In [1]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir)

In [2]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, user_knn as knn

In [3]:
import numpy as np
import pandas as pd
import math
%matplotlib inline

In [22]:
index=range(1,11)
#col_names1 = ["item", 'user']
col_names =  ['item', 'user', 'rank', 'protected', 'providers']
my_df  = pd.DataFrame(index = index, columns = col_names)

my_df["item"] = np.arange(1,11)
my_df["user"] = [1,1,1,1,1,1,1,1,1,1]
my_df["rank"] = np.arange(1,11)
my_df["protected"] = [0,0,0,1,0,0,0,0,1,1]
my_df["providers"] = ['a', 'b', 'c', 'c', 'c', 'c', 'c', 'c', 'c', 'c']


print(my_df)

                         
#data = np.array([np.arange(10)], [np.arange(10)])
#data.head

    item  user  rank  protected providers
1      1     1     1          0         a
2      2     1     2          0         b
3      3     1     3          0         c
4      4     1     4          1         c
5      5     1     5          0         c
6      6     1     6          0         c
7      7     1     7          0         c
8      8     1     8          0         c
9      9     1     9          1         c
10    10     1    10          1         c


In [23]:
items_dummy = pd.concat([my_df, my_df['providers'].str.get_dummies()], axis=1)
items_dummy.head()

,item,user,rank,protected,providers,a,b,c
1,1,1,1,0,a,1,0,0
2,2,1,2,0,b,0,1,0
3,3,1,3,0,c,0,0,1
4,4,1,4,1,c,0,0,1
5,5,1,5,0,c,0,0,1


In [24]:
res1= items_dummy.item.nunique()
print('res1:')
print(res1)
res2 =  items_dummy.loc[my_df['protected'] == 1]
print('res2:')
print(res2.item.nunique())
res3 = list(items_dummy.iloc[:,5:])
print('res3:')
print(res3)

rla = topnFair.FairRecListAnalysis(['user'])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
rla.add_metric("div")
#rla.add_metric("dem_parity")
results = rla.compute(my_df, my_df, 'protected', res3, items_N=res1, proItems_N=res2.item.nunique())
results.head()

res1:
10
res2:
3
res3:
['a', 'b', 'c']


KeyError: "['a' 'b' 'c'] not in index"

In [ ]:
#test:[0,0,0,1,0,0,0,0,1,1] non-protected
dmp_non_protected = (1/math.log2(1+1)) + (1/math.log2(1+2)) + (1/math.log2(1+3)) + (1/math.log2(1+5)) + (1/math.log2(1+6)) + (1/math.log2(1+7)) + (1/math.log2(1+8))


#test:[0,0,0,1,0,0,0,0,1,1] 
dmp_protected = (1/math.log2(1+4)) + (1/math.log2(1+9)) + (1/math.log2(1+10))

dmp_protected/dmp_non_protected

In [ ]:
#test rDN for my_df = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 0/10 vs 2/20 

rnd = (0.1/math.log(10+1,2)+(0/math.log(20+1,2))) 
rnd       


In [ ]:
#test rDN for my_df = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 2/10 vs 4/20 

dif1 = math.abs(0.0-0)
rnd1 = (0.1/math.log(10+1,2)+(0/math.log(20+1,2))) 
rnd1       

In [ ]:
#test rKL for my_df["protected"] = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1] 1/10 vs 4/20

px= 1/10
qx=4/20

rkl =(px*math.log(px/qx,2))+(1-px)*math.log((1-px)/(1-qx),2)

print(rkl)
print(rkl/math.log(11,2))

In [ ]:
#test rKL for my_df["protected"] = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1] 0/10 vs 2/20
# corner case -- px cant be 0 thus set manuale to 0,001

#px= 0/10 - px cant be 0 thus set manuale to 0,001
px = 0.001
qx=2/20

rkl2 =(px*math.log(px/qx,2))+(1-px)*math.log((1-px)/(1-qx),2)

print(rkl2)
print(rkl2/math.log(11,2))